**DRAFT script for scraping key data sets**

In [ ]:
## Import

import requests
from bs4 import BeautifulSoup
import pandas as pd 
import numpy as np
from datetime import date
import time


**Football-Data.co.uk**

In [ ]:
## Scraping football-data.co.uk

epl_odds = pd.read_csv('https://www.football-data.co.uk/mmz4281/2223/E0.csv')
epl_odds['scrape_dt'] = date.today()

epl_odds.head()


,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA,scrape_dt
0,E0,05/08/2022,20:00,Crystal Palace,Arsenal,0,2,A,0,1,...,0.50,2.09,1.84,2.04,1.88,2.09,1.88,2.03,1.85,2022-08-20
1,E0,06/08/2022,12:30,Fulham,Liverpool,2,2,D,1,0,...,1.75,1.90,2.03,1.91,2.02,2.01,2.06,1.89,1.99,2022-08-20
2,E0,06/08/2022,15:00,Bournemouth,Aston Villa,2,0,H,1,0,...,0.50,1.93,2.00,1.93,2.00,1.94,2.04,1.88,2.00,2022-08-20
3,E0,06/08/2022,15:00,Leeds,Wolves,2,1,H,1,1,...,-0.25,2.08,1.85,2.10,1.84,2.14,1.87,2.08,1.81,2022-08-20
4,E0,06/08/2022,15:00,Newcastle,Nott'm Forest,2,0,H,0,0,...,-1.00,1.97,1.96,1.99,1.93,2.19,1.97,2.03,1.86,2022-08-20


**SPI Data**

In [ ]:
## SPI data

spi_global_ranking  = pd.read_csv('https://projects.fivethirtyeight.com/soccer-api/club/spi_global_rankings.csv')
spi_matches = pd.read_csv('https://projects.fivethirtyeight.com/soccer-api/club/spi_matches.csv') ## historical data going back to 2016 inclusive of pre and post match metrics
spi_matches_latest = pd.read_csv('https://projects.fivethirtyeight.com/soccer-api/club/spi_matches_latest.csv')

spi_global_ranking['scrape_dt'] = date.today()


**FB Ref - B4Soup**

In [ ]:
## FB Ref
   ## https://www.sports-reference.com/bot-traffic.html Scraping policy
   ## Has xG data for EPL from 2018-18 onwards

# Setting URL
URL = "https://fbref.com/en/comps/9/2019-2020/schedule/2019-2020-Premier-League-Scores-and-Fixtures"

# HTTP request
page = requests.get(URL)

# Prints HTML
# print(page.content)

# Parsing the HTML using BeautifulSoup 
soup = BeautifulSoup(page.content, "html.parser")

# Printing a pretty version of HTML
#print(soup.prettify())

# Searching for table by ID
table_html = soup.find("div", {"id": "div_sched_2019-2020_9_1"})

#pd.read_html(table_html)

## https://medium.com/geekculture/web-scraping-tables-in-python-using-beautiful-soup-8bbc31c5803e
#df = pd.DataFrame(columns=['Week', 'Day', 'Home', 'xG_Home', 'Score', 'xG_Away', 'Away', 'Attendance', 'Venue', 'Referee', 'Match Report', 'Notes'])



**FB Ref - Pandas**

_Minimal Example_

In [ ]:
# https://pbpython.com/pandas-html-table.html

# Using Pandas to read in HTML of target URL
table_EPL = pd.read_html('https://fbref.com/en/comps/9/2019-2020/schedule/2019-2020-Premier-League-Scores-and-Fixtures')

## Checking the number of tables in URL
#print(f'Total tables: {len(table_EPL)}') - only 1 table

# Turning result into a data frame
df = table_EPL[0]
#df.head()

# Removing extra columns
df2 = df.iloc[:,0:12]
#df2.head()

# Splitting Score colum into two columns
df2[['Home_Score', 'Away_Score']] = df2['Score'].str.split('–', expand=True)

# Renaming xG columns
df2.rename(columns={'xG': 'xG_Home', 'xG.1': 'xG_Away'}, inplace=True)

df2




,Wk,Day,Date,Time,Home,xG_Home,Score,xG_Away,Away,Attendance,Venue,Referee,Home_Score,Away_Score
0,1.0,Fri,2019-08-09,20:00,Liverpool,1.7,4–1,1.0,Norwich City,53333.0,Anfield,Michael Oliver,4,1
1,1.0,Sat,2019-08-10,12:30,West Ham,0.8,0–5,3.1,Manchester City,59870.0,London Stadium,Mike Dean,0,5
2,1.0,Sat,2019-08-10,15:00,Crystal Palace,0.7,0–0,1.0,Everton,25151.0,Selhurst Park,Jonathan Moss,0,0
3,1.0,Sat,2019-08-10,15:00,Bournemouth,1.0,1–1,1.0,Sheffield Utd,10714.0,Vitality Stadium,Kevin Friend,1,1
4,1.0,Sat,2019-08-10,15:00,Burnley,0.7,3–0,0.8,Southampton,19784.0,Turf Moor,Graham Scott,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
414,38.0,Sun,2020-07-26,16:00,Chelsea,1.3,2–0,0.2,Wolves,NaN,Stamford Bridge,Stuart Attwell,2,0
415,38.0,Sun,2020-07-26,16:00,Newcastle Utd,0.4,1–3,0.6,Liverpool,NaN,St. James' Park,Anthony Taylor,1,3
416,38.0,Sun,2020-07-26,16:00,Leicester City,1.4,0–2,1.9,Manchester Utd,NaN,King Power Stadium,Martin Atkinson,0,2
417,38.0,Sun,2020-07-26,16:00,Manchester City,3.2,5–0,1.1,Norwich City,NaN,Etihad Stadium,Craig Pawson,5,0


**FB Ref - Pandas**

_Loop_

In [ ]:
# Reference - https://stackoverflow.com/questions/28669482/appending-pandas-dataframes-generated-in-a-for-loop


EPL_URLS = ['https://fbref.com/en/comps/9/2019-2020/schedule/2017-2018-Premier-League-Scores-and-Fixtures',
'https://fbref.com/en/comps/9/2019-2020/schedule/2018-2019-Premier-League-Scores-and-Fixtures',
'https://fbref.com/en/comps/9/2019-2020/schedule/2019-2020-Premier-League-Scores-and-Fixtures',
'https://fbref.com/en/comps/9/2020-2021/schedule/2020-2021-Premier-League-Scores-and-Fixtures',
'https://fbref.com/en/comps/9/2021-2022/schedule/2021-2022-Premier-League-Scores-and-Fixtures']


## Empty data frame
appended_data = []

for i in EPL_URLS:
  
  time.sleep(3) # Sleep for 3 seconds
  table_EPL = pd.read_html(i)
  df = table_EPL[0]
  # Subsetting columns
  df2 = df.iloc[:,0:12]
  # Splitting Score colum into two columns
  df2[['Home_Score', 'Away_Score']] = df2['Score'].str.split('–', expand=True)
  # Renaming xG columns
  df2.rename(columns={'xG': 'xG_Home', 'xG.1': 'xG_Away'}, inplace=True)

  # store DataFrame in list
  appended_data.append(df2)  





In [ ]:

type(appended_data)

list

In [ ]:
all_epl_data = pd.concat(appended_data)
all_epl_data

,Wk,Day,Date,Time,Home,xG_Home,Score,xG_Away,Away,Attendance,Venue,Referee,Home_Score,Away_Score
0,1.0,Fri,2019-08-09,20:00,Liverpool,1.7,4–1,1.0,Norwich City,53333.0,Anfield,Michael Oliver,4,1
1,1.0,Sat,2019-08-10,12:30,West Ham,0.8,0–5,3.1,Manchester City,59870.0,London Stadium,Mike Dean,0,5
2,1.0,Sat,2019-08-10,15:00,Crystal Palace,0.7,0–0,1.0,Everton,25151.0,Selhurst Park,Jonathan Moss,0,0
3,1.0,Sat,2019-08-10,15:00,Bournemouth,1.0,1–1,1.0,Sheffield Utd,10714.0,Vitality Stadium,Kevin Friend,1,1
4,1.0,Sat,2019-08-10,15:00,Burnley,0.7,3–0,0.8,Southampton,19784.0,Turf Moor,Graham Scott,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
443,38.0,Sun,2022-05-22,16:00,Norwich City,0.3,0–5,3.7,Tottenham,27022.0,Carrow Road,Chris Kavanagh,0,5
444,38.0,Sun,2022-05-22,16:00,Burnley,1.4,1–2,1.8,Newcastle Utd,21361.0,Turf Moor,Craig Pawson,1,2
445,38.0,Sun,2022-05-22,16:00,Liverpool,3.2,3–1,1.1,Wolves,53097.0,Anfield,Anthony Taylor,3,1
446,38.0,Sun,2022-05-22,16:00,Manchester City,2.9,3–2,0.3,Aston Villa,53395.0,Etihad Stadium,Michael Oliver,3,2
